In [74]:
import pandas as pd
import glob
import numpy as np

In [ ]:
def load_fer_output_data(path):
    csv_files = glob.glob(path + "/*.csv")
    csv_files.sort()

    df_list = (pd.read_csv(file) for file in csv_files)
    
    
    # Concatenate all DataFrames
    df = pd.concat(df_list, ignore_index=True)
    df.columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
    df['start']= [0.1*count for count in range(len(df))]
    df['end'] = [0.1*count for count in range(1,len(df)+1)]

    return df

In [64]:
def load_aggregated_fer_output_data(path):
    csv_files = glob.glob(path + "/*")
    csv_files.sort()

    labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
    df_list = [pd.read_csv(file).mean().to_list() for file in csv_files]
    
    df = pd.DataFrame(df_list, columns=['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'])
    df['start']= [5*count for count in range(len(df))]
    df['end'] = [5*count for count in range(1,len(df)+1)]

    return df

In [105]:
df_facial_vocals_morning = load_fer_output_data('../data/FER_Output/2_jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp/2_csv')
df_facial_vocals_morning = df_facial_vocals_morning.drop('end', axis=1)
df_facial_vocals_morning.columns = ['facial_anger', 'facial_disgust', 'facial_fear', 'facial_joy', 'facial_sadness', 'facial_surprise', 'facial_neutral', 'start_patch']
df_facial_vocals_morning

,facial_anger,facial_disgust,facial_fear,facial_joy,facial_sadness,facial_surprise,facial_neutral,start_patch
0,0.214425,0.000394,0.131845,0.240804,0.236482,0.008039,0.166783,0
1,0.315261,0.000390,0.131789,0.141539,0.214871,0.010243,0.184813,5
2,0.229541,0.000380,0.113441,0.211516,0.202054,0.014216,0.227643,10
3,0.185469,0.001125,0.187967,0.199728,0.263882,0.029651,0.130991,15
4,0.103115,0.000189,0.110050,0.193321,0.323976,0.019419,0.248989,20
5,0.128038,0.000704,0.101215,0.310280,0.254838,0.010516,0.193009,25
6,0.188894,0.001633,0.123617,0.222357,0.289280,0.009960,0.163117,30
7,0.147860,0.002095,0.129860,0.240620,0.306660,0.014920,0.156560,35
8,0.210137,0.001401,0.115640,0.172801,0.288479,0.011405,0.199185,40
9,0.188937,0.000605,0.131580,0.162400,0.307050,0.019716,0.188749,45


In [109]:
df_text_vocals_morning = pd.read_csv('../data/emotion_from_transcript/emotion-english-distilroberta-base/jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp.csv', index_col=0)
df_text_vocals_morning['start_patch'] = np.floor(((df_text_vocals_morning['start']/60)/5))*5
df_text_vocals_morning['start'] = np.floor((df_text_vocals_morning['start']/60))
df_text_vocals_morning = df_text_vocals_morning.groupby('start_patch')['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'].mean().reset_index()
df_text_vocals_morning.columns = ['start_patch', 'text_anger', 'text_disgust', 'text_fear', 'text_joy', 'text_sadness', 'text_surprise', 'text_neutral']


/tmp/ipykernel_3620/3011918079.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_text_vocals_morning = df_text_vocals_morning.groupby('start_patch')['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'].mean().reset_index()


In [122]:
df_vocals_morning_merged = pd.merge(df_text_vocals_morning, df_facial_vocals_morning, on=['start_patch'], how='inner').drop('start_patch', axis=1)

In [125]:
df_vocals_morning_corr = df_vocals_morning_merged.corr().drop(['text_anger', 'text_disgust', 'text_fear', 'text_joy', 'text_sadness', 'text_surprise', 'text_neutral'], axis=1)
df_vocals_morning_corr = df_vocals_morning_corr.drop(['facial_anger', 'facial_disgust', 'facial_fear', 'facial_joy', 'facial_sadness', 'facial_surprise', 'facial_neutral'],axis=0)
df_vocals_morning_corr

,facial_anger,facial_disgust,facial_fear,facial_joy,facial_sadness,facial_surprise,facial_neutral
text_anger,0.088131,-0.046638,-0.166281,-0.079025,0.032596,0.352811,-0.079247
text_disgust,-0.037573,-0.133239,-0.128602,0.218443,-0.005819,-0.116359,-0.048432
text_fear,0.136245,-0.338267,-0.332244,-0.136314,-0.360337,0.407742,0.314530
text_joy,0.036551,0.213681,0.436095,-0.045756,0.077573,-0.260738,-0.201377
text_sadness,-0.227920,0.429767,-0.056182,0.335701,0.168237,-0.068511,-0.160812
text_surprise,0.510384,-0.044646,0.179982,-0.226090,-0.188372,-0.008098,-0.266579
text_neutral,-0.286219,-0.221322,-0.272066,-0.028241,0.021881,0.135383,0.409433


In [2]:
df_text_vocals_afternoon = pd.read_csv('../data/emotion_from_transcript/emotion-english-distilroberta-base/jazzaar-20230417-afternoon-JFLB-Cuban-Vocals-video Philipp.csv', index_col=0)
df_text_vocals_morning = pd.read_csv('../data/emotion_from_transcript/emotion-english-distilroberta-base/jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp.csv', index_col=0)